# Mount FileDataset and extract video frames

In [1]:
!pip install --upgrade azureml-sdk azureml-dataprep opencv-python matplotlib

Requirement already up-to-date: azureml-sdk in /home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages (1.13.0)
Requirement already up-to-date: azureml-dataprep in /home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages (2.1.4)
Requirement already up-to-date: opencv-python in /home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages (4.4.0.42)
Requirement already up-to-date: matplotlib in /home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages (3.3.1)


In [2]:
from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
print(workspace)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataprep 2.1.4 (/home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages), Requirement.parse('azureml-dataprep<2.1.0a,>=2.0.1a'), {'azureml-dataset-runtime'}).
Workspace.create(name='aiml50dg2ai', subscription_id='6be80d16-1b1d-465b-bbdd-a298a0cbbf89', resource_group='mlops-demo-rg')


In [3]:
dataset_name="videos-samples"
dataset = Dataset.get_by_name(workspace, name=dataset_name)
print(dataset.name)

videos-samples


In [8]:
import cv2
import matplotlib.pyplot as plt
def processVideo(video_file_path):
     video_capture = cv2.VideoCapture(video_file_path)
     print(f"{video_file_path}: file opened {video_capture.isOpened()}")
     fps = int(video_capture.get(cv2.CAP_PROP_FPS))
     print(f"{video_file_path}: fps # {fps}")
     frame_count = 0
     video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
     success, frame = video_capture.read()
     print(f"{video_file_path}: {success} reading first frame")
     while success:
         temp_file_name = f"/tmp/video_frame_{frame_count:05d}.jpg"
         # convert to RGB
         rgb_frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
         plt.imsave(temp_file_name, rgb_frame)
         print(f"{frame_count:05d}: Saved frame in {temp_file_name}")
         frame_count += 1
         success, frame = video_capture.read()


In [10]:
# mount dataset manually instead of using
# with dataset.mount() as mount_context:
mount_context=dataset.mount()
mount_context.start()
# See where we mounted the file dataset
print(mount_context.mount_point)
# List files in there
print(os.listdir(mount_context.mount_point))


WARNING - Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataprep 2.1.4 (/home/azureuser/anaconda3/envs/wayve/lib/python3.8/site-packages), Requirement.parse('azureml-dataprep<2.1.0a,>=2.0.1a'), {'azureml-dataset-runtime'}).
/tmp/tmp5vsw2uqz
['bbb_sunflower_1080p_30fps_normal.mp4']


In [11]:

# Loop through files
for f in os.listdir(mount_context.mount_point):
    if os.path.splitext(f)[-1] == '.mp4': # Process only files ending in mp4
        full_file_path = os.path.join(mount_context.mount_point, f)
        processVideo(full_file_path)


/tmp/tmp5vsw2uqz/bbb_sunflower_1080p_30fps_normal.mp4: file opened True
/tmp/tmp5vsw2uqz/bbb_sunflower_1080p_30fps_normal.mp4: fps # 30
/tmp/tmp5vsw2uqz/bbb_sunflower_1080p_30fps_normal.mp4: True reading first frame
00000: Saved frame in /tmp/video_frame_00000.jpg
00001: Saved frame in /tmp/video_frame_00001.jpg
00002: Saved frame in /tmp/video_frame_00002.jpg
00003: Saved frame in /tmp/video_frame_00003.jpg
00004: Saved frame in /tmp/video_frame_00004.jpg
00005: Saved frame in /tmp/video_frame_00005.jpg
00006: Saved frame in /tmp/video_frame_00006.jpg
00007: Saved frame in /tmp/video_frame_00007.jpg
00008: Saved frame in /tmp/video_frame_00008.jpg
00009: Saved frame in /tmp/video_frame_00009.jpg
00010: Saved frame in /tmp/video_frame_00010.jpg
00011: Saved frame in /tmp/video_frame_00011.jpg
00012: Saved frame in /tmp/video_frame_00012.jpg
00013: Saved frame in /tmp/video_frame_00013.jpg
00014: Saved frame in /tmp/video_frame_00014.jpg
00015: Saved frame in /tmp/video_frame_00015.jpg


KeyboardInterrupt: 

In [7]:
# Unmount dataset
mount_context.stop()